### Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import os
import datetime

### Reading Raw data files

In [2]:
df_SIR_counts = pd.read_csv(filepath_or_buffer="D:/Covid - 19/06-04-2020/covid_19_india.csv")
df_hospital_beds = pd.read_csv(filepath_or_buffer="D:/Covid - 19/06-04-2020/HospitalBedsIndia.csv")
df_population = pd.read_csv(filepath_or_buffer="D:/Covid - 19/06-04-2020/population_india_census2011.csv")

### Variable Treatment for SIR Counts Data

In [3]:
# Converting to datetime format
df_SIR_counts['Date'] = pd.to_datetime(df_SIR_counts['Date'],format="%d/%m/%y")

# Replave "-" by nan and convert to numeric
df_SIR_counts.loc[(df_SIR_counts.ConfirmedForeignNational == '-'),'ConfirmedForeignNational'] = np.nan
df_SIR_counts.loc[(df_SIR_counts.ConfirmedIndianNational == '-'),'ConfirmedIndianNational'] = np.nan

# Converting to numeric columns
df_SIR_counts.ConfirmedForeignNational = pd.to_numeric(df_SIR_counts.ConfirmedForeignNational)
df_SIR_counts.ConfirmedIndianNational = pd.to_numeric(df_SIR_counts.ConfirmedIndianNational)

# changing "Chattisgarh" to "Chhattisgarh"
df_SIR_counts.loc[(df_SIR_counts['State/UnionTerritory'] == 'Chattisgarh'),'State/UnionTerritory'] = "Chhattisgarh"

# Sort by State and Date
df_SIR_counts = df_SIR_counts.sort_values(by=['State/UnionTerritory','Date'])

### Filtering and Merging Population data

In [4]:
# Sort by State
df_population = df_population[["State / Union Territory","Population"]].sort_values(by = 'State / Union Territory')

# Rename to match the column name in SIR counts data
df_population = df_population.rename(columns={"State / Union Territory":"State/UnionTerritory"})

# Merging Population and SIR Count
df_ADS = pd.merge(left=df_SIR_counts,right=df_population,how='inner',on=['State/UnionTerritory'])

# Calculating Susceptibles and removed
df_ADS['Removed'] = df_ADS['Cured'] + df_ADS['Deaths']
df_ADS['Susceptibles'] = df_ADS['Population'] - (df_ADS['Confirmed'] + df_ADS['Removed'])

### Filtering and Merging Hospital Beds Data

In [5]:
# Rename State/UT 
df_hospital_beds = df_hospital_beds.rename(columns={"State/UT" : "State/UnionTerritory"})
# filter for bed count Columns
df_hospital_beds = df_hospital_beds[['State/UnionTerritory','NumPublicBeds_HMIS','NumUrbanBeds_NHP18','NumRuralBeds_NHP18']]
# Drop rows which have all NaN values
df_hospital_beds = df_hospital_beds.dropna(thresh=3,axis=0)
# Select data for only states
df_hospital_beds = df_hospital_beds.iloc[1:35]
# COnvert to numeric
df_hospital_beds['NumPublicBeds_HMIS'] = pd.to_numeric(df_hospital_beds['NumPublicBeds_HMIS'])
# Calculate total number of beds
df_hospital_beds['Total_beds'] = df_hospital_beds['NumPublicBeds_HMIS'] + df_hospital_beds['NumRuralBeds_NHP18'] + df_hospital_beds['NumUrbanBeds_NHP18']
# Filer columns for merging
df_hospital_beds = df_hospital_beds[['State/UnionTerritory','Total_beds']]
# Merging hospital beds to the ADS
df_ADS = pd.merge(left=df_ADS,right=df_hospital_beds,how='left',on=['State/UnionTerritory'])
# Find number of beds available
df_ADS['Total_Available_Beds'] = df_ADS['Total_beds'] - df_ADS['Confirmed']

In [6]:
# Creating the lock down flag
df_ADS['Lockdown_Flag'] = np.where(df_ADS.Date > "2020-03-24",1,0)

In [7]:
df_ADS.describe()

,Sno,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,Population,Removed,Susceptibles,Total_beds,Total_Available_Beds,Lockdown_Flag
count,675.000000,446.000000,446.000000,675.000000,675.000000,675.000000,6.750000e+02,675.000000,6.750000e+02,552.000000,552.000000,675.000000
mean,338.567407,12.188341,1.495516,2.605926,0.764444,33.648889,4.562126e+07,3.370370,4.562122e+07,62621.601449,62585.436594,0.496296
std,195.771121,21.582253,3.576292,6.400943,2.100426,67.509201,4.641721e+07,7.823859,4.641719e+07,45387.790779,45365.247117,0.500357
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,2.740000e+05,0.000000,2.739760e+05,3989.000000,3987.000000,0.000000
25%,169.500000,1.000000,0.000000,0.000000,0.000000,3.000000,1.226703e+07,0.000000,1.226703e+07,25081.000000,25029.500000,0.000000
50%,338.000000,3.000000,0.000000,0.000000,0.000000,8.000000,3.340606e+07,0.000000,3.340606e+07,66979.000000,66875.000000,0.000000
75%,507.500000,13.000000,1.000000,2.500000,1.000000,32.000000,6.854844e+07,3.000000,6.854837e+07,83937.000000,83901.250000,1.000000
max,678.000000,177.000000,14.000000,49.000000,24.000000,503.000000,1.998123e+08,66.000000,1.998123e+08,150148.000000,150147.000000,1.000000


In [8]:
df_ADS.head()

,Sno,Date,Time,State/UnionTerritory,ConfirmedIndianNational,ConfirmedForeignNational,Cured,Deaths,Confirmed,Population,Removed,Susceptibles,Total_beds,Total_Available_Beds,Lockdown_Flag
0,366,2020-03-26,6:00 PM,Andaman and Nicobar Islands,1.0,0.0,0,0,1,380581,0,380580,NaN,NaN,1
1,393,2020-03-27,10:00 AM,Andaman and Nicobar Islands,1.0,0.0,0,0,1,380581,0,380580,NaN,NaN,1
2,421,2020-03-28,6:00 PM,Andaman and Nicobar Islands,6.0,0.0,0,0,6,380581,0,380575,NaN,NaN,1
3,448,2020-03-29,7:30 PM,Andaman and Nicobar Islands,NaN,NaN,0,0,9,380581,0,380572,NaN,NaN,1
4,475,2020-03-30,9:30 PM,Andaman and Nicobar Islands,NaN,NaN,0,0,9,380581,0,380572,NaN,NaN,1


### Writing it in a CSV

In [10]:
df_ADS.to_csv("D:/Covid - 19/06-04-2020/ADS.csv",index = False)